1\. Write a function that converts number representation (bin<->dec<->hex)

In [2]:
def hex_to_dec(number):
	print('your hexadecimal number in decimal is:',int(number))

 #emter your hexadeciml number starting with 0x

number=0xa2d4
hex_to_dec(number)


your hexadecimal number in decimal is: 41684


In [3]:
def bin_to_dec(number):
    print(' your binary number in decimal is :', int(number))
 
 #enter your binary number starting with 0b 

number=0b101001
bin_to_dec(number)


 your binary number in decimal is : 41


In [4]:
def hex_to_bin(number):
	dec=int(number)
	print('your hexadecimal number in binary is:',bin(dec))
    
 #enter your hexadeciml number starting with 0x

number=0xf53a9e
hex_to_bin(number)


your hexadecimal number in binary is: 0b111101010011101010011110


In [ ]:
def dec_to_bin_and_hex(dec):
	print(dec,'in binary is:',bin(dec))
	print(dec,'in hexadecimal is:',hex(dec))
    
dec=int(input('give me yoor decimal number'))

dec_to_bin_and_hex(dec)

In [ ]:
def bin_to_hex(number):
	dec=int(number)
	print(' your binary number in hexadecimal is:',hex(dec))
    
 #enter your binary number starting with 0b 

number=0b10110101010
bin_to_hex(number)

2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [ ]:
def WORD(word):
    word=list(word)
    s=int(word[0])
    ex=''.join(word[1:9])
    m=''.join(word[9:33])
    ex=int(ex,2)
    m=int(m,2)
    a=float('1.'+ str(m))
    print ('the decimal float number is:',((-1)**s)*(a)*((2)**(ex-127)))
    

word=input('give me your 32 bit word') 
WORD(word)

3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [ ]:
 #finding underflow limit (my laptop's underflow is 4e-324)
a=1
while (a!=0) :
    b=a
    a=a/2
print(b)


In [ ]:
 #finding the overflow limit (my laptop's over flow is veryyyyy big! )
a=1
b=2
while (a<b):
    a=a+1
    b=b*2
print(b)



4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [ ]:
 # detemining the precision of my machine

i = 0
precision= 1
while((float(0.1)+precision)!=float(0.1)) :
    precision = precision/2
    i=i+1

print(precision,i)

5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [ ]:
import math 
def root1(a,b,c):
    r1=(((-b)+math.sqrt((b*b)-4*a*c))/2*a)
    r2=(((-b)-math.sqrt((b*b)+4*a*c))/2*a)
    print('the first root is:' ,r1,'   and    the secon root is:',r2)
    
root(0.001,1000,0.001)

In [ ]:
import math 
def root2(a,b,c):
    r1=(((-b)-math.sqrt((b*b)-4*a*c)))
    r2=(((-b)+math.sqrt((b*b)+4*a*c)))
    print('the first root is:' ,r1,'   and    the secon root is:',r2)
    
root(0.001,1000,0.001)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [11]:
def func(x):
    return (x*(x-1))

def delta_derivation(x,delta):
    df=((func(x+delta)-func(x))/delta)
    print(df)
    
x=1
delta=10**(-2)
delta_derivation(x,delta)
DELTA=[10**-4,10**-6,10**-8,10**-10,10**-12,10**-14] 
for delta in DELTA :
    delta_derivation(x,delta)

    

1.010000000000001
1.0000999999998899
1.0000009999177333
1.0000000039225287
1.000000082840371
1.0000889005833413
0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [ ]:
import math

def func(x):
    return(math.sqrt(1-x**2))

def integral(N) :
    domain=[-1 + i*(2/N) for i in range(N)]
    y=[(2/N)*func(d) for d in domain]
    return (sum(y))

N=100
integral(N)
%timeit print('if we take N=100 our error will be:',(((math.pi)/2)-integral(N)))

      
N=10000000
integral(N)
%timeit print('if we take N=10000000, our error will be :',(((math.pi)/2)-integral(N)))   



if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

...

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1

if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=100 our error will be: 0.0016620712456461018
if we take N=1